In [ ]:
pip install txtai

In [ ]:
pip install txtai\[pipeline-llm\]

In [1]:
from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor, LLM

/opt/anaconda3/envs/envo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/envo/lib/python3.8/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_list" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/envs/envo/lib/python3.8/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/envs/envo/lib/python3.8/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_group_alias" has conflict with protected namespace "model_".

Yo

In [2]:
QA = [
["original: Given the objects dataset, how many types of objects can be found, according to their volume?",
"rewrited: Load dataset, calculate volume, cluster, count cluster"],
["original: Train a model that can estimate density based on surface and mass given the objects dataset.",
"rewrited: Load dataset, calculate density, split dataset, perform regression density using surface and mass, calculate MSE"],

["original: Calculate the regression of mass using height in the objects dataset, and test the model.",
"rewrited: Load dataset, split dataset, perform regression mass using height, calculate MSE"],

["original: Train a model to estimate density using surface and mass data from the object dataset.",
"rewrited: Load dataset, calculate density, split dataset, perform regression density using surface and mass, calculate MSE"],

["original: Determine the variety of objects in the dataset based on their volume.",
"rewrited: Load dataset, calculate volume, cluster, count clusters"],

["original: Calculate the regression of mass using height within the dataset of objects.",
"rewrited: Load dataset, split dataset, perform regression mass using height, calculate MSE"],

["original: How many clusters can be identified within the dataset based on height?",
"rewrited: Load dataset, cluster using height, count clusters"],

["original: Calculate the density of objects using their volume and mass from the dataset.",
"rewrited: Load dataset, calculate density"],

["original: Identify different types of objects based on volume and surface area.",
"rewrited: Load dataset, calculate volume, cluster using volume and surface"],

["original: What is the volume for all objects within the dataset?",
"rewrited: Load dataset, calculate volume"],

["original: How many objects in the dataset have a specific ID?",
"rewrited: Load dataset, count using ID"],

["original: Predict volume based on height through regression analysis.",
"rewrited: Load dataset, calculate volume, split dataset, perform regression volume using height, calculate MSE"],

["original: Cluster objects based on height and mass within the dataset.",
"rewrited: Load dataset, cluster using height and mass"],

["original: Estimate volume using surface and height through regression.",
"rewrited: Load dataset, calculate volume, split dataset, perform regression density using surface and mass, calculate MSE"],


["original: How many clusters can be identified within the dataset based on volume?",
"rewrited: Load dataset, calculate volume, cluster, count clusters"]
]


In [3]:
embeddings = Embeddings(content=True, autoid="uuid5")

In [4]:
embeddings.index(QA)

In [5]:
llm = LLM("TheBloke/Mistral-7B-OpenOrca-GGUF/mistral-7b-openorca.Q4_K_M.gguf", verbose=False)

In [6]:
template = """<|im_start|>system
You are an assistant that will rewrite sentences from users using a specific vocabulary.<|im_end|>
<|im_start|>user
Find the best matching text in the context for the input, try to find something similar in the "original" part of the context. 
The response should only be taken from the corresponding "rewrited" part from the context, you can slightly adapt if needed (e.g., attribute names).

Question:
{question}

Context:
{context}

Text:
<|im_end|>
<|im_start|>assistant
"""

In [7]:
extractor = Extractor(embeddings, llm, output="reference", separator="\n", template=template)

In [8]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [9]:
result = extractor("Predict height based on volume through regression.")

In [10]:
print("ANSWER:", result["answer"])
print("REFERENCE:", embeddings.search("select id, text from txtai where id = :id", parameters={"id": result["reference"]}))


ANSWER: The response should only be taken from the corresponding "rewrited" part from the context. In this case, you can slightly adapt the text to find something similar in the rewritten part of the context.

Text: Predict height based on volume through regression by loading a dataset, calculating volume, splitting it into training and testing sets, performing regression using height as the independent variable, and then calculating the mean squared error (MSE).
REFERENCE: [{'id': 'aecd8629-2764-503d-b842-11aba0efd57e', 'text': 'original: Predict volume based on height through regression analysis. rewrited: Load dataset, calculate volume, split dataset, perform regression volume using height, calculate MSE'}]


In [11]:
result2 = extractor("How many different types of objects based on volume and height.")

In [12]:
print("ANSWER:", result2["answer"])
print("REFERENCE:", embeddings.search("select id, text from txtai where id = :id", parameters={"id": result2["reference"]}))


ANSWER: How many different types of objects can be found, according to their volume? Load dataset, calculate volume, cluster, count cluster.
REFERENCE: [{'id': 'ff07167c-2e01-5ca4-bab2-6633f70d6c6c', 'text': 'original: Given the objects dataset, how many types of objects can be found, according to their volume? rewrited: Load dataset, calculate volume, cluster, count cluster'}]


In [13]:
result7 = extractor("Train a model that can estimate volume based on surface given the objects dataset.")
print("ANSWER:", result7["answer"])
print("REFERENCE:", embeddings.search("select id, text from txtai where id = :id", parameters={"id": result7["reference"]}))

ANSWER: Train a model that can estimate volume based on surface given the objects dataset.
REFERENCE: [{'id': 'f94bc3c6-754d-5f51-a1ed-530c235cbceb', 'text': 'original: Train a model that can estimate density based on surface and mass given the objects dataset. rewrited: Load dataset, calculate density, split dataset, perform regression density using surface and mass, calculate MSE'}]


In [14]:
result3 = extractor("Identify different types of objects based on volume and height.")
print("ANSWER:", result3["answer"])
print("REFERENCE:", embeddings.search("select id, text from txtai where id = :id", parameters={"id": result3["reference"]}))

ANSWER: In the context, you want to identify different types of objects based on their volume and height. The best matching text would be from the "rewrited" part: Load dataset, calculate volume, and cluster using volume and surface area (since height is not explicitly mentioned).
REFERENCE: [{'id': '0365f368-b6e9-5483-8f63-7add0d8fe7ad', 'text': 'original: Identify different types of objects based on volume and surface area. rewrited: Load dataset, calculate volume, cluster using volume and surface'}]


In [15]:
result4 = extractor("Determine the density of objects using volume and mass data.")
print("ANSWER:", result4["answer"])
print("REFERENCE:", embeddings.search("select id, text from txtai where id = :id", parameters={"id": result4["reference"]}))

ANSWER: Determine the density of objects using volume and mass data.
REFERENCE: [{'id': '4b3abf9f-9e1f-522b-8b4b-6052cf3cb209', 'text': 'original: Calculate the density of objects using their volume and mass from the dataset. rewrited: Load dataset, calculate density'}]


In [16]:
result5 = extractor("Cluster objects based on volume and surface area within the dataset.")
print("ANSWER:", result5["answer"])
print("REFERENCE:", embeddings.search("select id, text from txtai where id = :id", parameters={"id": result5["reference"]}))

ANSWER: Cluster objects based on volume and surface area within the dataset.
REFERENCE: [{'id': '0365f368-b6e9-5483-8f63-7add0d8fe7ad', 'text': 'original: Identify different types of objects based on volume and surface area. rewrited: Load dataset, calculate volume, cluster using volume and surface'}]


In [17]:
result6 = extractor("Estimate height using volume and mass through regression analysis.")
print("ANSWER:", result6["answer"])
print("REFERENCE:", embeddings.search("select id, text from txtai where id = :id", parameters={"id": result6["reference"]}))

ANSWER: Find the best matching text in the context for the input: "Estimate height using volume and mass through regression analysis."

Response: Load dataset, split dataset, perform regression density using surface and mass, calculate MSE
REFERENCE: [{'id': 'bf6e0a17-03fb-5b2d-ba08-1a6f71148aae', 'text': 'original: Estimate volume using surface and height through regression. rewrited: Load dataset, calculate volume, split dataset, perform regression density using surface and mass, calculate MSE'}]
